In [11]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [12]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 15   # size of group 1
ny = 5    # size of group 2
MC = 10_000
perms = nothing
if binomial(nx + ny, nx) < 30_000
    perms = partition(nx, ny)
end

([1 1 … 5 6; 2 2 … 7 7; … ; 14 14 … 19 19; 15 16 … 20 20], [20 20 … 6 5; 19 19 … 4 4; … ; 17 17 … 2 2; 16 15 … 1 1])

In [13]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = LogNormal{dtype}
X_mu = random(Uniform(0, 1), B)
X_sigma = random(Uniform(0, 0.6), B)
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Gamma{dtype}
Y_shape = @. 1 / (exp(X_sigma^2) - 1)
Y_scale = @. exp(X_mu + X_sigma^2 / 2) / Y_shape
distrY = map(distrTypeY, Y_shape, Y_scale);

In [14]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = LogNormal{Float32}[LogNormal{Float32}(μ=0.906f0, σ=0.572f0), LogNormal{Float32}(μ=0.443f0, σ=0.508f0)]
distrY[1:2] = Gamma{Float32}[Gamma{Float32}(α=2.5836008f0, θ=1.1279569f0), Gamma{Float32}(α=3.3964891f0, θ=0.5216751f0)]
deltas[1:2] = Float32[0.0, 0.0]


In [15]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


5×4300 Matrix{Float32}:
 2.44324   1.97657  0.592417  2.05831   …  1.18617  3.5021    2.1959
 0.839079  1.62085  1.83155   5.96646      2.88642  0.992534  3.4971
 2.03561   2.59261  0.586244  1.59166      1.03209  1.47581   1.6656
 0.492746  1.22292  0.686977  0.636792     3.4526   1.5387    1.89366
 1.13457   1.35263  1.28554   1.11607      1.4801   3.98515   2.90282

In [16]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

5×4300 Matrix{Float32}:
 2.44324   1.97657  0.592417  2.05831   …  1.18617  3.5021    2.1959
 0.839079  1.62085  1.83155   5.96646      2.88642  0.992534  3.4971
 2.03561   2.59261  0.586244  1.59166      1.03209  1.47581   1.6656
 0.492746  1.22292  0.686977  0.636792     3.4526   1.5387    1.89366
 1.13457   1.35263  1.28554   1.11607      1.4801   3.98515   2.90282

In [17]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [18]:
T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
 
    results[1, b, s] = permInterval(x, y, deltas[b], perms, true, alpha, twoSided, twoSided, 0.0005)    
    results[2, b, s] = permInterval(x, y, deltas[b], perms, false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(x, y, deltas[b], perms, true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], perms, false, alpha/2, greater, smaller, 0.0005)
    
    """
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/1/"
save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 6, B, S; prefix="bs", dir=dir)

1357.115033 seconds (3.39 G allocations: 50.887 TiB, 50.06% gc time, 0.51% compilation time)
dir * filename = "../results/15_5/1/twoSided_pooled_0.05.csv"
dir * filename = "../results/15_5/1/twoSided_unpooled_0.05.csv"
dir * filename = "../results/15_5/1/oneSided_pooled_0.025.csv"
dir * filename = "../results/15_5/1/oneSided_unpooled_0.025.csv"
